### konlpy를 사용한 형태소분리와 Word2Vec를 사용한 벡터변환

In [3]:
# # 한국어 전처리를 위한
!pip install konlpy
# !pip install Twitter
# !pip install tqdm
!pip install gensim
# !pip install --upgrade jupyter ipywidgets


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models.word2vec import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악
import re #  Python의 정규 표현식(regular expression) 모듈 -> re.sub() 함수를 사용하여 문자열에서 정규 표현식 패턴에 일치하는 부분을 다른 문자열로 대체
import html # html 엔터티 코드 제거를 위해 사용

In [5]:
# 본인 컴퓨터와 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [6]:
# 데이터를 데이터프레임으로 로드
musical_data = pd.read_csv('../mock_test_csv/mock.csv', encoding='utf-8')# 데이터 파일 경로
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,NaN,뮤지컬 <실연남녀> 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 ...,어리버리 조폭 형제가 경영하는 “아트 퓨쳐산장”. 이들은 6개월 전 “아트 퓨쳐산장...
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,90분,“No.1 창작뮤지컬의 신화”뮤지컬 ‘김종욱찾기(연출/김지연)’는 창작뮤지컬계 블루...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디“운명은 항상...
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,130분,"뮤지컬 <첫사랑>은 어느 바닷가를 배경으로, 이루어지지 못한 첫사랑에 대한 아련한 ...",시간을 조금만 되돌릴 수 있다면…어릴 적부터 한 동네에서 자란 해수와 선이는 오누이...
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬!2003년 초연 이래로 16번의 ...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다. 하지만 희...
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,160분 (인터미션 : 20분),3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...
...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/11/06 ~ 2008/02/03,충무아트센터 소극장 블루,"김우형,윤공주,문성혁,임은영,문지원",7세이상,110분,컨/페/션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑...,그대 가슴속에 두고 와야 할 게 있습니다…그건 바로 사/랑/ 입니다피아노 연주와 신...
96,5270,비보이 퍼포먼스 마리오네트,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/01/12 ~ 2007/03/11,충무아트센터 소극장 블루,NaN,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인!!!타 기존 작품들과의 차별화 성공!2006년 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...
97,5299,마리아 마리아,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/08/23 ~ 2007/08/26,충무아트센터 대극장,"강효성,김보강,이승철",만 8세이상,NaN,2007 뮤지컬 마리아마리아는..뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없...,"ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자, 제사장들은 이에 위..."
98,6509,어린이뮤지컬 장화신은 고양이,http://ticketimage.interpark.com/PlayDictionar...,어린이/가족,2007/07/28 ~ 2007/08/19,국립중앙박물관 극장 용,NaN,만 5세이상,60분,러시아 푸슈킨(Pushkin) 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여...,NaN


In [7]:
# 모든 열에 대해 특수 문자와 HTML 엔터티 코드 모두 제거
for col in musical_data.columns:
    musical_data[col] = musical_data[col].apply(lambda x: re.sub(r'[^\w\s]', '', html.unescape(str(x))))
# 결과 출력
print(musical_data)

   musical_id            title  \
0        3885             실연남녀   
1        3635           김종욱 찾기   
2        4829              첫사랑   
3        3322              미라클   
4        4424              대장금   
..        ...              ...   
95       5075              컨페션   
96       5270   비보이 퍼포먼스 마리오네트   
97       5299          마리아 마리아   
98       6509  어린이뮤지컬 장화신은 고양이   
99       1162            빨간모자2   

                                           poster_url  genre  \
0   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
1   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
2   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
3   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
4   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
..                                                ...    ...   
95  httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
96  httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
97  httpticketi

In [8]:
musical_data.to_csv('../cleaned_data_mock.csv', index=False, encoding='utf-8')

### 텍스트 전처리

In [9]:
# 형태소 분리를 위한 Konlpy 객체 초기화
twitter = Okt()

In [10]:
#텍스트를 전처리 하는 함수
#BOOK_INTRCN_CN: 전처리할 도서 줄거리
def preprocessingText(synopsis):
    #전처리한 결과를 저장 할 리스트
    stems = []
    #text 를 단어로 나누고 품사를 붙여서 리턴
    #stem=True : 단어의 기본형으로 변환  ex) 같습니다,같았습니다 -> 같다.  
    tagged_review = twitter.pos(synopsis,stem=True)

    #tagged에서 하나를 꺼내서 word,pos에 대입
    #단어는 word, 품사는 pos에 대입
    for word,pos in tagged_review:
        #len(word) : 단어의 글자수가 2글자 이상이면
        #품사가 Noun(명사) 또는 Adjective(형용사) 일경우에만 word(단어) 를 stems에 추가
        if pos=="Noun" or pos=='Adjective':
            stems.append(word)

    #리스트인 stems를 문자열로 만들어서 ( "".join(stems) ) 리턴       
    return " ".join(stems)

In [11]:
preprocessingText(musical_data['synopsis'][0])

'조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마비 산장 주인 때문 난감하다 그 은행강도 사건 수배 중인 데 다가 자신 산장 주인 범인 오인 수도 있다 때문 우선 산장 주인 시체 그 대신 산장 운영 새롭다 삶 시작 맹세 그 후 개월 후 손님 교통사고 취소 난감하다 차 여자 손님 지아 남자 손님 연오 손님 좋아하다 그것 잠시 이 두 사람 자살 위해 이 산장 방문 것 알 난감하다 다애 실연 지아 배신 복수 위해 아내 죽음 연오 슬픔 위해 두 사람 추억 장소 아트 퓨쳐 산장 산장 자살 것 목 연오 지아 이 자살 장면 목격 운수 재수 결사 두 사람 만약 이 자살 시체 발견 경찰 수사 것 자신 전적 두렵다 것 자살 실패하다 지아 연오 또 다시 자살 위해 애 운수 재수 지아 연오 자살 위해 온갖 방법 원한 그렇다 가운데 지아 연오 사랑 때문 입장 것 알 그 아픔 이해 연오 역시 지아 아픔 마음 이해 그 연합전선 형성 자신 자살 운수 재수 형제 대항 온갖 죽 방법 원한 마침내 운수 재수 산장 밖 죽 기회 지아 연오 산장 밖 이 것 보고 발 동동 운수 재수 마침내 운수 특단 조치 생각 그 자살 위해 재수 그 비법 과연 이 그 밤 그 원하다 것 손 것 자 자 한바탕 소동 자살 방지 프로젝트 시작'

In [12]:
# 데이터프레임의 'BOOK_INTRCN_CN' 열에 NaN 값이 있을 때 빈 문자열로 대체 및 전처리
musical_data['synopsis_clear'] = musical_data['synopsis'].fillna('').apply(preprocessingText)

In [13]:
# Word2Vec 모델 훈련을 위한 데이터 준비
sentences = musical_data['synopsis'].tolist()

In [14]:
# 데이터프레임의 'synopsis' 열에 NaN 값이 있을 때 빈 문자열로 대체
tokenized_data = musical_data['synopsis'].apply(lambda x: preprocessingText(str(x))).fillna('')

In [15]:
tokenized_data

0     조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...
1     첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...
2     시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...
3     인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...
4     어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...
                            ...                        
95    그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...
96    이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...
97    예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...
98                                                     
99    앤 할머니 시간 뒤 과거 뒤 기차 추억 역 향 추억 역 도착 앤 할머니 마침내 뒤 ...
Name: synopsis, Length: 100, dtype: object

In [16]:
#BOOK_INTRCN_CN_clear의 마지막에 " " 추가
musical_data["synopsis_clear"] = musical_data['synopsis_clear'].astype(str) + " "

In [17]:
#data.split(" ") : 문자열을 공백기준으로 잘라서 리스트로 변환
musical_data["synopsis_clear"].apply(lambda data : data.split(" "))

0     [조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...
1     [첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...
2     [시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...
3     [인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...
4     [어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...
                            ...                        
95    [그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...
96    [이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...
97    [예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...
98                                                 [, ]
99    [앤, 할머니, 시간, 뒤, 과거, 뒤, 기차, 추억, 역, 향, 추억, 역, 도착...
Name: synopsis_clear, Length: 100, dtype: object

In [18]:
#df["BOOK_INTRCN_CN_clear"].apply(lambda data : data.split(" ")) : BOOK_INTRCN_CN_clear 컬럼을 공백을 기준으로 단어별로 분리해서 리턴
musical_data["tokenized_data"] = musical_data["synopsis_clear"].apply(lambda data : data.split(" "))

In [19]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,..."
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,..."
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]"


In [20]:
musical_data["tokenized_data"]

0     [조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...
1     [첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...
2     [시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...
3     [인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...
4     [어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...
                            ...                        
95    [그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...
96    [이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...
97    [예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...
98                                                 [, ]
99    [앤, 할머니, 시간, 뒤, 과거, 뒤, 기차, 추억, 역, 향, 추억, 역, 도착...
Name: tokenized_data, Length: 100, dtype: object

### 유사도 측정
- 특정 줄거리와 다른 모든 줄거리 간의 유사도를 측정

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import random

In [22]:
# 학습용 Word2Vec 모델 훈련(벡터화)
model = Word2Vec(musical_data["tokenized_data"], 
                 vector_size=100, #WordVector의 열의 수
                 window=3, # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=2, # 사용할 단어의 최소 빈도(2회 이하 단어 무시)
                 sg=1) # 0: CBOW, 1: Skip-gram

In [23]:
model.save("word2vec_model2.bin")

In [24]:
#Vector 형태로 변환한 단어 리스트 조회
model.wv.key_to_index

{'그': 0,
 '있다': 1,
 '자신': 2,
 '것': 3,
 '사랑': 4,
 '': 5,
 '그녀': 6,
 '수': 7,
 '위해': 8,
 '없다': 9,
 '사람': 10,
 '이': 11,
 '샌디': 12,
 '예수': 13,
 '말': 14,
 '대니': 15,
 '다시': 16,
 '이야기': 17,
 '집': 18,
 '마리아': 19,
 '존': 20,
 '새롭다': 21,
 '친구': 22,
 '고양이': 23,
 '킴': 24,
 '모습': 25,
 '마음': 26,
 '시작': 27,
 '일': 28,
 '남자': 29,
 '속': 30,
 '서로': 31,
 '렌': 32,
 '날': 33,
 '노래': 34,
 '크리스': 35,
 '그렇다': 36,
 '여자': 37,
 '같다': 38,
 '엄마': 39,
 '둘': 40,
 '춤': 41,
 '하나': 42,
 '때': 43,
 '더': 44,
 '곳': 45,
 '에스메랄다': 46,
 '아이': 47,
 '도시': 48,
 '막': 49,
 '알': 50,
 '한편': 51,
 '생각': 52,
 '삶': 53,
 '마을': 54,
 '안': 55,
 '온': 56,
 '꿈': 57,
 '다른': 58,
 '전': 59,
 '사이': 60,
 '파티': 61,
 '등': 62,
 '두': 63,
 '아름답다': 64,
 '모든': 65,
 '무대': 66,
 '아버지': 67,
 '토니': 68,
 '후': 69,
 '앞': 70,
 '살': 71,
 '순간': 72,
 '클럽': 73,
 '나': 74,
 '모두': 75,
 '자기': 76,
 '공연': 77,
 '소리': 78,
 '어머니': 79,
 '때문': 80,
 '대한': 81,
 '뮤지컬': 82,
 '계속': 83,
 '간다': 84,
 '시간': 85,
 '중': 86,
 '새': 87,
 '학교': 88,
 '음악': 89,
 '사실': 90,
 '생활': 91,
 '소개': 92,
 '듀터': 

In [25]:
#믿음의 인덱스(빈도수 순위 0이 가장 빈도수 높은 단어)
model.wv.key_to_index["할머니"]

508

In [26]:
#믿음과 비슷한 벡터를 가진 단어 조회
model.wv.most_similar("할머니")

[('온', 0.9903969764709473),
 ('그', 0.9901576042175293),
 ('생활', 0.9900235533714294),
 ('이', 0.9899621605873108),
 ('순간', 0.9899594187736511),
 ('뮤지컬', 0.9899246692657471),
 ('없다', 0.9898408055305481),
 ('등', 0.9896857142448425),
 ('못', 0.989596962928772),
 ('자살', 0.9895666837692261)]

In [27]:
#믿음의 WordVector 조회
model.wv["할머니"]

array([-0.01433351,  0.04204848, -0.00544856, -0.01216272,  0.02901947,
       -0.11127229,  0.03458539,  0.09163379, -0.02381896, -0.02000765,
        0.01640509, -0.04261837, -0.02161255,  0.03493305,  0.01145365,
       -0.00841854,  0.02013972, -0.08701663, -0.00152325, -0.07279485,
        0.04109769, -0.00450007,  0.04680617, -0.05608162, -0.00515776,
       -0.00316679, -0.05660516, -0.02925156, -0.03373524,  0.01149199,
        0.08969821,  0.02235052,  0.00843773, -0.03236167,  0.02547478,
        0.06719057,  0.01068659, -0.06010433, -0.02756315, -0.07407408,
       -0.03894145, -0.0243636 , -0.04368816,  0.00035104,  0.04240602,
       -0.03500374, -0.02151581,  0.00053144,  0.00892226,  0.03489668,
        0.00815805, -0.03384209, -0.03204204, -0.01628654, -0.01330847,
        0.02469962,  0.04109024,  0.00068758, -0.07453973,  0.00126794,
        0.00207071, -0.00491362, -0.02420492,  0.0223962 , -0.08125868,
        0.02533285,  0.00473266,  0.02466785, -0.05793021,  0.07

In [28]:
#WordVector로 변환한 단어들 word2vec_words 대입
word2vec_words = model.wv.key_to_index.keys()
word2vec_words

dict_keys(['그', '있다', '자신', '것', '사랑', '', '그녀', '수', '위해', '없다', '사람', '이', '샌디', '예수', '말', '대니', '다시', '이야기', '집', '마리아', '존', '새롭다', '친구', '고양이', '킴', '모습', '마음', '시작', '일', '남자', '속', '서로', '렌', '날', '노래', '크리스', '그렇다', '여자', '같다', '엄마', '둘', '춤', '하나', '때', '더', '곳', '에스메랄다', '아이', '도시', '막', '알', '한편', '생각', '삶', '마을', '안', '온', '꿈', '다른', '전', '사이', '파티', '등', '두', '아름답다', '모든', '무대', '아버지', '토니', '후', '앞', '살', '순간', '클럽', '나', '모두', '자기', '공연', '소리', '어머니', '때문', '대한', '뮤지컬', '계속', '간다', '시간', '중', '새', '학교', '음악', '사실', '생활', '소개', '듀터', '노미', '희망', '리조', '행동', '눈', '위', '아니다', '인생', '듀티율', '이제', '궁', '목사', '자살', '못', '밤', '이름', '추억', '동안', '모어', '록시', '또', '자리', '콰', '통해', '손님', '번', '좋아하다', '도착', '가지', '최고', '관계', '점점', '대해', '과연', '모도', '작품', '댄스', '엔지니어', '내', '감정', '주인', '버몬', '장금', '거리', '딸', '향', '벽', '세상', '현실', '어떻다', '일상', '뒤', '역시', '많다', '파의', '젤리', '인하다', '이용', '메리', '소냐', '끝', '분위기', '돈', '캣츠비', '결혼', '마지막', '채', '찰리', '여인', '운명', '페뷔스', '아들', '선', '오해', '지선', 

In [29]:
word2vec_words

dict_keys(['그', '있다', '자신', '것', '사랑', '', '그녀', '수', '위해', '없다', '사람', '이', '샌디', '예수', '말', '대니', '다시', '이야기', '집', '마리아', '존', '새롭다', '친구', '고양이', '킴', '모습', '마음', '시작', '일', '남자', '속', '서로', '렌', '날', '노래', '크리스', '그렇다', '여자', '같다', '엄마', '둘', '춤', '하나', '때', '더', '곳', '에스메랄다', '아이', '도시', '막', '알', '한편', '생각', '삶', '마을', '안', '온', '꿈', '다른', '전', '사이', '파티', '등', '두', '아름답다', '모든', '무대', '아버지', '토니', '후', '앞', '살', '순간', '클럽', '나', '모두', '자기', '공연', '소리', '어머니', '때문', '대한', '뮤지컬', '계속', '간다', '시간', '중', '새', '학교', '음악', '사실', '생활', '소개', '듀터', '노미', '희망', '리조', '행동', '눈', '위', '아니다', '인생', '듀티율', '이제', '궁', '목사', '자살', '못', '밤', '이름', '추억', '동안', '모어', '록시', '또', '자리', '콰', '통해', '손님', '번', '좋아하다', '도착', '가지', '최고', '관계', '점점', '대해', '과연', '모도', '작품', '댄스', '엔지니어', '내', '감정', '주인', '버몬', '장금', '거리', '딸', '향', '벽', '세상', '현실', '어떻다', '일상', '뒤', '역시', '많다', '파의', '젤리', '인하다', '이용', '메리', '소냐', '끝', '분위기', '돈', '캣츠비', '결혼', '마지막', '채', '찰리', '여인', '운명', '페뷔스', '아들', '선', '오해', '지선', 

### 벡터를 문자열로 저장

In [30]:
#행의 수를 row_num에 저장
row_num = len(musical_data)
row_num

100

In [31]:
string_array = []  # 문자열을 저장할 빈 리스트

for index in range(row_num):
    # 뮤지컬 번호 조회
    NUM = musical_data.loc[index,"musical_id"]
    print("NUM=",NUM)
    # 뮤지컬 제목 조회
    TITLE = musical_data.loc[index,"title"]
    print("TITLE=",TITLE)
    # 줄거리 조회
    LINE = musical_data.loc[index,"tokenized_data"]
    print("LINE=",LINE)
    doc2vec = None
    count = 0
    for word in LINE: #줄거리에서 단어 1개씩 리턴
        print("word=",word, end="\t")
        if word in word2vec_words: #단어가 WordVector에 포함된 단어 이면
            count += 1 #개수 1증가
            if doc2vec is None: #doc2vect이 None(초기값 맨처음 실행 첫번째 단어) 이면
                doc2vec = model.wv[word] #단어의 WordVector 값 대입
            else: #첫번째 단어 아님
                doc2vec = doc2vec + model.wv[word] #단어의 WordVector 값 추가

    if  doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count
     # 문자열로 변환해서 string_array 리스트에 추가
    string_array.append(doc2vec.tostring())

# string_array에 저장된 문자열을 데이터프레임에 추가
musical_data["doc2vec_vec"] = string_array

NUM= 3885
TITLE= 실연남녀
LINE= ['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월', '전', '아트', '퓨쳐', '산장', '사채', '빚', '갑자기', '심장마비', '산장', '주인', '때문', '난감하다', '그', '은행강도', '사건', '수배', '중인', '데', '다가', '자신', '산장', '주인', '범인', '오인', '수도', '있다', '때문', '우선', '산장', '주인', '시체', '그', '대신', '산장', '운영', '새롭다', '삶', '시작', '맹세', '그', '후', '개월', '후', '손님', '교통사고', '취소', '난감하다', '차', '여자', '손님', '지아', '남자', '손님', '연오', '손님', '좋아하다', '그것', '잠시', '이', '두', '사람', '자살', '위해', '이', '산장', '방문', '것', '알', '난감하다', '다애', '실연', '지아', '배신', '복수', '위해', '아내', '죽음', '연오', '슬픔', '위해', '두', '사람', '추억', '장소', '아트', '퓨쳐', '산장', '산장', '자살', '것', '목', '연오', '지아', '이', '자살', '장면', '목격', '운수', '재수', '결사', '두', '사람', '만약', '이', '자살', '시체', '발견', '경찰', '수사', '것', '자신', '전적', '두렵다', '것', '자살', '실패하다', '지아', '연오', '또', '다시', '자살', '위해', '애', '운수', '재수', '지아', '연오', '자살', '위해', '온갖', '방법', '원한', '그렇다', '가운데', '지아', '연오', '사랑', '때문', '입장', '것', '알', '그', '아픔', '이해', '연오', '역시', '지아', '아픔', '마음', '이해', '그', '연합전선', '형성', '자신', '자살', 

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_18156\2134378797.py:28: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  string_array.append(doc2vec.tostring())


In [32]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\x85#\x07\xbd\x10\xbcf=\xcb:\x9f;\xc6|N\xbc\...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b""\xcf+\x04\xbd\x8d\x13m=c\xbe\xf7:\x9ae(\xbc\..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'u\x96\x0f\xbd\x014\x80=h\x87\x91;\xa0e!\xbco...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...","b""j\x8d\x1a\xbd\xc6\x98\x8a=B\x83j;\x9aCP\xbc\..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...","b'\xb2F=\xbd\x93\x05\xa7=u\xa1\x88;N""_\xbc\xcb..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xb7\xf6\x18\xbdo\xc4\x84=hu&;\xd98-\xbc\xa7...
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...","b""\x1cF \xbd\x16#\x95=+\xbf\x1e;\\-7\xbcM\x8c*..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]","b""\xa4\xed~\xbdT?\xc5=\x1e$\x8b;\xd19\xdb\xbc\..."


#### date를 start_date와 end_date로 분리

In [33]:
# 날짜 형식에 맞게 잘라서 "YYYY-MM-DD" 또는 "YYYY-MM" 형식의 문자열로 만듭니다.
musical_data['start_date'] = musical_data['date'].str[:4] + '-' + musical_data['date'].str[4:6] + '-' + musical_data['date'].str[6:8]
musical_data['end_date'] = musical_data['date'].str[:4] + '-' + musical_data['date'].str[4:6]

In [34]:
# datetime 객체로 변환
musical_data['start_date'] = pd.to_datetime(musical_data['start_date'], format="%Y-%m-%d")
musical_data['end_date'] = pd.to_datetime(musical_data['end_date'], format="%Y-%m")

In [35]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec,start_date,end_date
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc...",2007-10-13,2007-10-01
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\x85#\x07\xbd\x10\xbcf=\xcb:\x9f;\xc6|N\xbc\...,2006-12-12,2006-12-01
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b""\xcf+\x04\xbd\x8d\x13m=c\xbe\xf7:\x9ae(\xbc\...",2007-03-27,2007-03-01
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'u\x96\x0f\xbd\x014\x80=h\x87\x91;\xa0e!\xbco...,2007-06-01,2007-06-01
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...","b""j\x8d\x1a\xbd\xc6\x98\x8a=B\x83j;\x9aCP\xbc\...",2007-05-26,2007-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...","b'\xb2F=\xbd\x93\x05\xa7=u\xa1\x88;N""_\xbc\xcb...",2007-11-06,2007-11-01
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xb7\xf6\x18\xbdo\xc4\x84=hu&;\xd98-\xbc\xa7...,2007-01-12,2007-01-01
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...","b""\x1cF \xbd\x16#\x95=+\xbf\x1e;\\-7\xbcM\x8c*...",2007-08-23,2007-08-01
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]","b""\xa4\xed~\xbdT?\xc5=\x1e$\x8b;\xd19\xdb\xbc\...",2007-07-28,2007-07-01


In [36]:
# 'date'와 'location' 사이로 'start_date'와 'end_date' 위치 변경
musical_data = musical_data[['musical_id', 'title', 'poster_url', 'genre', 'date', 'start_date', 'end_date', 'location',
                             'actors', 'age_rating', 'running_time', 'describe', 'synopsis', 'synopsis_clear',
                             'tokenized_data', 'doc2vec_vec']]
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\x85#\x07\xbd\x10\xbcf=\xcb:\x9f;\xc6|N\xbc\...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b""\xcf+\x04\xbd\x8d\x13m=c\xbe\xf7:\x9ae(\xbc\..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'u\x96\x0f\xbd\x014\x80=h\x87\x91;\xa0e!\xbco...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...","b""j\x8d\x1a\xbd\xc6\x98\x8a=B\x83j;\x9aCP\xbc\..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...","b'\xb2F=\xbd\x93\x05\xa7=u\xa1\x88;N""_\xbc\xcb..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xb7\xf6\x18\xbdo\xc4\x84=hu&;\xd98-\xbc\xa7...
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...","b""\x1cF \xbd\x16#\x95=+\xbf\x1e;\\-7\xbcM\x8c*..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,2007-07-28,2007-07-01,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]","b""\xa4\xed~\xbdT?\xc5=\x1e$\x8b;\xd19\xdb\xbc\..."


In [37]:
# # musical_data.to_csv('../musical_data_vector.csv', index=False, encoding='utf-8') # 벡터변환 csv저장
# musical_data.to_csv('../mock_test_csv/musical_data_vector2.csv', index=False, encoding='utf-8')

### 추천 시스템

In [38]:
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\x85#\x07\xbd\x10\xbcf=\xcb:\x9f;\xc6|N\xbc\...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b""\xcf+\x04\xbd\x8d\x13m=c\xbe\xf7:\x9ae(\xbc\..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'u\x96\x0f\xbd\x014\x80=h\x87\x91;\xa0e!\xbco...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...","b""j\x8d\x1a\xbd\xc6\x98\x8a=B\x83j;\x9aCP\xbc\..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...","b'\xb2F=\xbd\x93\x05\xa7=u\xa1\x88;N""_\xbc\xcb..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xb7\xf6\x18\xbdo\xc4\x84=hu&;\xd98-\xbc\xa7...
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...","b""\x1cF \xbd\x16#\x95=+\xbf\x1e;\\-7\xbcM\x8c*..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,2007-07-28,2007-07-01,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]","b""\xa4\xed~\xbdT?\xc5=\x1e$\x8b;\xd19\xdb\xbc\..."


In [39]:
#wordvector을 문자열로 변환 한 doc2vec_str 컬럼 값을 string_array에 저장
string_array = musical_data.loc[0,"doc2vec_vec"]
string_array

b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc\xce\xefC=\xa3\xb6R\xbeJ\xccl=/\xc2E>\xabEi\xbd\x85\xe1\xb5\xbc\xc4P\x9e<.\xb6\xa3\xbds\xba\xc2\xbc\xdcm\x91=l\x04\x04=\x18>\xc4\xbc\xbc[d=(\xb5\x1f\xbe4b\x17\xbcvi\x02\xbe%|\x93=\xd9S\xbe;\x9f\xf5\xcc=&\x11\xcb\xbd\xa1L\'\xbbp\x83\\;G\xb0\xbe\xbd$[\x87\xbd\xc4\x0fN\xbd\x83\x89\x9e<\x99\x97)>\x00\xc7\xc0<,\x1b\xe3<\xf1\xcc\x8e\xbd\xe6\xe6\x12=\xfdx\r>\xfd\x81\n<\xe7\x10\xc4\xbdM\x82f\xbdl\xa9\x05\xbe\xfc\x05\x8b\xbda\xbau\xbdo\x95\xac\xbd\xa6\xe2\x9c\xbb\xb7T\xb9=fH\x81\xbd1@\x10\xbd\x94\xb29\xbb\xd4\x9c\x06<\x0c^\xa2=\xde\x90\xe9<\xff\x90\x84\xbd\xaar\x86\xbd\x11]P\xbcY)[\xbcj\xad\x83=\x14"\x94=fa9<t\x13\x0b\xbe\x1a-\xc4\xbb\xe8`\xa5\xbbM[\xd7\xbc\x14R5\xbd80^=\xa1O\x1c\xbe\xee^r=\xead\xb7\xbbD\xf9Z=\x19M\xca\xbd.\xf0\t>\x1c$\x19\xbd\xcc\xe7\x18=\xa1\x04\xa0=tX\xd3<,\x8c\x81=J\xb5\xf1<\xc7\xcf\x8f<\xf2m\x17\xbd\xe58\x95\xbd\xbf\x1es=\\\xb6\x0c\xbd\x84\x157=\x85\xad\x8e\xbd\xa6\x0f\x0e>\xaa\x18\x0f\xbd\x0c\xfa\x0f\xbcf\x12x\xbc8Z\xb3=H\xc8\

In [40]:
# wordvector을 문자열로 변환한  string_array 를 다시 numpy 배열로 변환
np.fromstring(string_array,dtype="float32")

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_18156\1195417262.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np.fromstring(string_array,dtype="float32")


array([-0.04213336,  0.07669467,  0.00537622, -0.0132969 ,  0.04783612,
       -0.20577483,  0.05781201,  0.19312356, -0.05695121, -0.02220226,
        0.01932562, -0.07993732, -0.02377055,  0.07101032,  0.03223078,
       -0.02395539,  0.05575155, -0.1559645 , -0.00923972, -0.12735543,
        0.07201413,  0.00580834,  0.10007786, -0.0991538 , -0.00255279,
        0.00336477, -0.09310966, -0.06609181, -0.050308  ,  0.01935268,
        0.16561736,  0.02353239,  0.02772292, -0.06972683,  0.03586473,
        0.13815685,  0.00845384, -0.09573536, -0.05627661, -0.1305291 ,
       -0.06788251, -0.0599922 , -0.0842694 , -0.00478776,  0.09049361,
       -0.06312637, -0.03521747, -0.00283352,  0.0082161 ,  0.07928094,
        0.02851146, -0.06472968, -0.06564839, -0.0127175 , -0.01337656,
        0.0642956 ,  0.07233062,  0.01131473, -0.1358164 , -0.00598682,
       -0.00504695, -0.02628865, -0.04426773,  0.0542452 , -0.15264751,
        0.05917256, -0.00559675,  0.05346037, -0.09877986,  0.13

In [41]:
 # wordvector을 문자열로 변환한  string_array 를 다시 numpy 배열로 변환한 결과를 numpy_array에 대입
numpy_array = np.fromstring(string_array,dtype="float32")
numpy_array

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_18156\2740756532.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  numpy_array = np.fromstring(string_array,dtype="float32")


array([-0.04213336,  0.07669467,  0.00537622, -0.0132969 ,  0.04783612,
       -0.20577483,  0.05781201,  0.19312356, -0.05695121, -0.02220226,
        0.01932562, -0.07993732, -0.02377055,  0.07101032,  0.03223078,
       -0.02395539,  0.05575155, -0.1559645 , -0.00923972, -0.12735543,
        0.07201413,  0.00580834,  0.10007786, -0.0991538 , -0.00255279,
        0.00336477, -0.09310966, -0.06609181, -0.050308  ,  0.01935268,
        0.16561736,  0.02353239,  0.02772292, -0.06972683,  0.03586473,
        0.13815685,  0.00845384, -0.09573536, -0.05627661, -0.1305291 ,
       -0.06788251, -0.0599922 , -0.0842694 , -0.00478776,  0.09049361,
       -0.06312637, -0.03521747, -0.00283352,  0.0082161 ,  0.07928094,
        0.02851146, -0.06472968, -0.06564839, -0.0127175 , -0.01337656,
        0.0642956 ,  0.07233062,  0.01131473, -0.1358164 , -0.00598682,
       -0.00504695, -0.02628865, -0.04426773,  0.0542452 , -0.15264751,
        0.05917256, -0.00559675,  0.05346037, -0.09877986,  0.13

In [42]:
#모든행의 doc2vec_vec 컬럼을 출력
musical_data.loc[: ,"doc2vec_vec"]

0     b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc...
1     b'\x85#\x07\xbd\x10\xbcf=\xcb:\x9f;\xc6|N\xbc\...
2     b"\xcf+\x04\xbd\x8d\x13m=c\xbe\xf7:\x9ae(\xbc\...
3     b'u\x96\x0f\xbd\x014\x80=h\x87\x91;\xa0e!\xbco...
4     b"j\x8d\x1a\xbd\xc6\x98\x8a=B\x83j;\x9aCP\xbc\...
                            ...                        
95    b'\xb2F=\xbd\x93\x05\xa7=u\xa1\x88;N"_\xbc\xcb...
96    b'\xb7\xf6\x18\xbdo\xc4\x84=hu&;\xd98-\xbc\xa7...
97    b"\x1cF \xbd\x16#\x95=+\xbf\x1e;\\-7\xbcM\x8c*...
98    b"\xa4\xed~\xbdT?\xc5=\x1e$\x8b;\xd19\xdb\xbc\...
99    b'LM!\xbd\xe3\xb6\x8e=%\xae\x81;\x87\xdae\xbc\...
Name: doc2vec_vec, Length: 100, dtype: object

In [43]:
##모든행의 doc2vec_vec 컬럼을 numpy 배열로 변환
musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_18156\1483775256.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )


0     [-0.042133365, 0.076694675, 0.0053762197, -0.0...
1     [-0.03299286, 0.056331694, 0.0048593036, -0.01...
2     [-0.03226834, 0.057879973, 0.0018901344, -0.01...
3     [-0.035055596, 0.06259919, 0.0044411905, -0.00...
4     [-0.03773252, 0.067674205, 0.0035783802, -0.01...
                            ...                        
95    [-0.04621, 0.0815536, 0.004169638, -0.01361901...
96    [-0.03734466, 0.06482779, 0.002539957, -0.0105...
97    [-0.03912936, 0.07282083, 0.0024222832, -0.011...
98    [-0.06223835, 0.096312195, 0.004246249, -0.026...
99    [-0.039380357, 0.069684766, 0.003957527, -0.01...
Name: doc2vec_vec, Length: 100, dtype: object

In [44]:
##모든행의 doc2vec_vec 컬럼을 numpy 배열로 변환해서 doc2vec_numpy 컬럼에 대입
musical_data.loc[: ,"doc2vec_numpy"] = musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_18156\2972792042.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  musical_data.loc[: ,"doc2vec_numpy"] = musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )
C:\Users\dsmdm\AppData\Local\Temp\ipykernel_18156\2972792042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musical_data.loc[: ,"doc2vec_numpy"] = musical_data.loc[: ,"doc2vec_vec"].apply(lambda x: np.fromstring(x,dtype="float32") )


In [45]:
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec,doc2vec_numpy
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc...","[-0.042133365, 0.076694675, 0.0053762197, -0.0..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\x85#\x07\xbd\x10\xbcf=\xcb:\x9f;\xc6|N\xbc\...,"[-0.03299286, 0.056331694, 0.0048593036, -0.01..."
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b""\xcf+\x04\xbd\x8d\x13m=c\xbe\xf7:\x9ae(\xbc\...","[-0.03226834, 0.057879973, 0.0018901344, -0.01..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'u\x96\x0f\xbd\x014\x80=h\x87\x91;\xa0e!\xbco...,"[-0.035055596, 0.06259919, 0.0044411905, -0.00..."
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...","b""j\x8d\x1a\xbd\xc6\x98\x8a=B\x83j;\x9aCP\xbc\...","[-0.03773252, 0.067674205, 0.0035783802, -0.01..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...","b'\xb2F=\xbd\x93\x05\xa7=u\xa1\x88;N""_\xbc\xcb...","[-0.04621, 0.0815536, 0.004169638, -0.01361901..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xb7\xf6\x18\xbdo\xc4\x84=hu&;\xd98-\xbc\xa7...,"[-0.03734466, 0.06482779, 0.002539957, -0.0105..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...","b""\x1cF \xbd\x16#\x95=+\xbf\x1e;\\-7\xbcM\x8c*...","[-0.03912936, 0.07282083, 0.0024222832, -0.011..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,2007-0

In [46]:
# doc2vec_numpy 배열의 값을 리스트로 변환
musical_data["doc2vec_numpy"].tolist()

[array([-0.04213336,  0.07669467,  0.00537622, -0.0132969 ,  0.04783612,
        -0.20577483,  0.05781201,  0.19312356, -0.05695121, -0.02220226,
         0.01932562, -0.07993732, -0.02377055,  0.07101032,  0.03223078,
        -0.02395539,  0.05575155, -0.1559645 , -0.00923972, -0.12735543,
         0.07201413,  0.00580834,  0.10007786, -0.0991538 , -0.00255279,
         0.00336477, -0.09310966, -0.06609181, -0.050308  ,  0.01935268,
         0.16561736,  0.02353239,  0.02772292, -0.06972683,  0.03586473,
         0.13815685,  0.00845384, -0.09573536, -0.05627661, -0.1305291 ,
        -0.06788251, -0.0599922 , -0.0842694 , -0.00478776,  0.09049361,
        -0.06312637, -0.03521747, -0.00283352,  0.0082161 ,  0.07928094,
         0.02851146, -0.06472968, -0.06564839, -0.0127175 , -0.01337656,
         0.0642956 ,  0.07233062,  0.01131473, -0.1358164 , -0.00598682,
        -0.00504695, -0.02628865, -0.04426773,  0.0542452 , -0.15264751,
         0.05917256, -0.00559675,  0.05346037, -0.0

In [47]:
#doc2vec_numpy 배열의 값을 리스트로 변환 한 값을 numpy 배열로 변환
np.array(musical_data["doc2vec_numpy"].tolist() )

array([[-0.04213336,  0.07669467,  0.00537622, ..., -0.14613767,
        -0.00628023, -0.02629839],
       [-0.03299286,  0.05633169,  0.0048593 , ..., -0.10658333,
        -0.00508031, -0.02124526],
       [-0.03226834,  0.05787997,  0.00189013, ..., -0.11200369,
        -0.00608764, -0.01918608],
       ...,
       [-0.03912936,  0.07282083,  0.00242228, ..., -0.13490297,
        -0.00567512, -0.0224125 ],
       [-0.06223835,  0.0963122 ,  0.00424625, ..., -0.1869243 ,
        -0.00550214, -0.03922491],
       [-0.03938036,  0.06968477,  0.00395753, ..., -0.12671332,
        -0.00485813, -0.02350982]], dtype=float32)

In [48]:
from sklearn.preprocessing import StandardScaler
#데이터의 각 열의 평균을 뺀 다음 표준편차로 나눠서 평균을 0로 표준편차를 1로 변환하는 StandardScaler 객체 생성
scaler = StandardScaler()

In [49]:
#각 열의 평균 표준편차 계산
scaler.fit(np.array(musical_data["doc2vec_numpy"].tolist() ))

StandardScaler()

In [50]:
#각 열의 평균
scaler.mean_

array([-0.04400949,  0.07689425,  0.00406703, -0.01471318,  0.04679375,
       -0.20602443,  0.05695681,  0.19381089, -0.05749825, -0.02260822,
        0.01746296, -0.08171065, -0.02370253,  0.07092901,  0.03045032,
       -0.02481845,  0.05411178, -0.15572966, -0.00888679, -0.12747432,
        0.07181453,  0.00504144,  0.10035549, -0.0983669 , -0.00224711,
        0.00367747, -0.09225799, -0.06771293, -0.05116964,  0.01960856,
        0.1664178 ,  0.02215692,  0.02916522, -0.07157447,  0.03502216,
        0.13756228,  0.00950262, -0.09641658, -0.05523258, -0.12970036,
       -0.06670563, -0.06290673, -0.08645007, -0.00593096,  0.09093106,
       -0.06291184, -0.03472369, -0.00186045,  0.00981108,  0.07803788,
        0.02967816, -0.06542578, -0.06471334, -0.01157591, -0.01485925,
        0.06366058,  0.07174393,  0.00846779, -0.13668039, -0.00512185,
       -0.00492364, -0.02632956, -0.04218161,  0.05436399, -0.15256162,
        0.06147852, -0.00550912,  0.05357005, -0.09892972,  0.13

In [51]:
#각 열의 분산
scaler.var_

array([7.80772221e-05, 1.67853133e-04, 1.43443317e-06, 2.01183032e-05,
       5.39535673e-05, 1.17954523e-03, 1.02105570e-04, 1.07561712e-03,
       1.07311527e-04, 2.47540717e-05, 6.48682670e-06, 2.00533461e-04,
       2.78134084e-05, 1.26479161e-04, 2.25166229e-05, 3.42962576e-05,
       7.11500294e-05, 7.14058176e-04, 4.19789413e-06, 4.67701249e-04,
       1.21085456e-04, 1.17970047e-06, 3.15567458e-04, 2.41817466e-04,
       6.03740819e-06, 1.64342362e-06, 2.25665183e-04, 1.52952964e-04,
       7.50715843e-05, 1.98222794e-05, 7.89431815e-04, 1.45572220e-05,
       2.01405494e-05, 1.74082842e-04, 3.28779508e-05, 5.78434748e-04,
       7.71182683e-06, 2.46040619e-04, 8.23321003e-05, 4.82389822e-04,
       1.07831065e-04, 1.30683019e-04, 2.37316924e-04, 2.15255542e-06,
       2.30104589e-04, 1.21587850e-04, 3.39051639e-05, 6.11007623e-06,
       3.85289053e-06, 1.64212760e-04, 3.54876231e-05, 1.41937217e-04,
       1.01555711e-04, 6.47259460e-06, 2.10527247e-05, 1.17754426e-04,
      

In [52]:
#데이터의 각 열의 평균을 뺀 다음 표준편차로 나눠서 평균을 0로 표준편차를 1로 변환 한 후 리스트로 변환해서 doc2vec_numpy_scale 컬럼에 대입
musical_data["doc2vec_numpy_scale"] = scaler.transform(np.array(musical_data["doc2vec_numpy"].tolist() )).tolist()

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_18156\1926048544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musical_data["doc2vec_numpy_scale"] = scaler.transform(np.array(musical_data["doc2vec_numpy"].tolist() )).tolist()


In [53]:
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_vec,doc2vec_numpy,doc2vec_numpy_scale
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...","b'\t\x94,\xbd\x19\x12\x9d=\x00+\xb0;;\xdbY\xbc...","[-0.042133365, 0.076694675, 0.0053762197, -0.0...","[0.21232469379901886, -0.015403996221721172, 1..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\x85#\x07\xbd\x10\xbcf=\xcb:\x9f;\xc6|N\xbc\...,"[-0.03299286, 0.056331694, 0.0048593036, -0.01...","[1.2467713356018066, -1.5871292352676392, 0.66..."
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...","b""\xcf+\x04\xbd\x8d\x13m=c\xbe\xf7:\x9ae(\xbc\...","[-0.03226834, 0.057879973, 0.0018901344, -0.01...","[1.328766107559204, -1.4676246643066406, -1.81..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'u\x96\x0f\xbd\x014\x80=h\x87\x91;\xa0e!\xbco...,"[-0.035055596, 0.06259919, 0.0044411905, -0.00...","[1.013327717781067, -1.103369951248169, 0.3124..."
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...","b""j\x8d\x1a\xbd\xc6\x98\x8a=B\x83j;\x9aCP\xbc\...","[-0.03773252, 0.067674205, 0.0035783802, -0.01...","[0.7103758454322815, -0.7116527557373047, -0.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...","b'\xb2F=\xbd\x93\x05\xa7=u\xa1\x88;N""_\xbc\xcb...","[-0.04621, 0.0815536, 0.004169638, -0.01361901...","[-0.2490348517894745, 0.35963425040245056, 0.0..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xb7\xf6\x18\xbdo\xc4\x84=hu&;\xd98-\xbc\xa7...,"[-0.03734466, 0.06482779, 0.002539957, -0.0105...","[0.7542704343795776, -0.931354284286499, -1.27..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아

In [54]:
# doc2vec_numpy_scale 컬럼 조회
musical_data["doc2vec_numpy_scale"] 

0     [0.21232469379901886, -0.015403996221721172, 1...
1     [1.2467713356018066, -1.5871292352676392, 0.66...
2     [1.328766107559204, -1.4676246643066406, -1.81...
3     [1.013327717781067, -1.103369951248169, 0.3124...
4     [0.7103758454322815, -0.7116527557373047, -0.4...
                            ...                        
95    [-0.2490348517894745, 0.35963425040245056, 0.0...
96    [0.7542704343795776, -0.931354284286499, -1.27...
97    [0.5522928237915039, -0.314408540725708, -1.37...
98    [-2.062990665435791, 1.4987825155258179, 0.149...
99    [0.5238872766494751, -0.5564667582511902, -0.0...
Name: doc2vec_numpy_scale, Length: 100, dtype: object

In [55]:
from sklearn.metrics.pairwise import euclidean_distances
# doc2vec_numpy_scale 컬럼의 유클리드 거리를 계산
sim_score = euclidean_distances(musical_data["doc2vec_numpy_scale"].tolist(),musical_data["doc2vec_numpy_scale"].tolist()) 

In [56]:
sim_score

array([[0.00000000e+00, 1.40439278e+01, 1.40653715e+01, ...,
        6.39176932e+00, 1.74306277e+01, 7.61080134e+00],
       [1.40439278e+01, 6.30796268e-07, 4.31263205e+00, ...,
        1.06625634e+01, 2.81599482e+01, 7.64105788e+00],
       [1.40653715e+01, 4.31263205e+00, 5.84003864e-07, ...,
        1.04544058e+01, 2.80378307e+01, 7.45553350e+00],
       ...,
       [6.39176932e+00, 1.06625634e+01, 1.04544058e+01, ...,
        1.19209290e-07, 2.08134609e+01, 5.75664499e+00],
       [1.74306277e+01, 2.81599482e+01, 2.80378307e+01, ...,
        2.08134609e+01, 4.76837158e-07, 2.20880597e+01],
       [7.61080134e+00, 7.64105788e+00, 7.45553350e+00, ...,
        5.75664499e+00, 2.20880597e+01, 2.92001932e-07]])

In [57]:
# sim_score (synopsys_vector_numpy_scale 컬럼의 유클리드 거리) DataFrame 으로 변환
sim_df = pd.DataFrame(data=sim_score)

In [58]:
sim_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,1.404393e+01,1.406537e+01,1.145351e+01,8.358102e+00,4.583631,5.386287,3.948157,4.868180,14.675750,...,11.657546,16.399346,1.743063e+01,5.807932,5.930404,3.769239e+00,1.011557e+01,6.391769e+00,1.743063e+01,7.610801e+00
1,14.043928,6.307963e-07,4.312632e+00,4.948234e+00,7.113935e+00,11.734876,9.899838,16.248913,11.034139,3.090800,...,4.936666,4.718702,2.815995e+01,8.924507,9.474908,1.593553e+01,5.673473e+00,1.066256e+01,2.815995e+01,7.641058e+00
2,14.065371,4.312632e+00,5.840039e-07,4.519653e+00,6.787456e+00,11.504501,9.867438,15.908099,10.985998,3.159820,...,3.817372,5.055598,2.803783e+01,9.027842,9.068932,1.577140e+01,5.091188e+00,1.045441e+01,2.803783e+01,7.455533e+00
3,11.453511,4.948234e+00,4.519653e+00,1.685874e-07,5.014768e+00,9.322559,7.754122,13.568989,8.308853,4.808602,...,3.812373,6.561628,2.571213e+01,6.625292,7.063287,1.324164e+01,3.496508e+00,8.409672e+00,2.571213e+01,5.223513e+00
4,8.358102,7.113935e+00,6.787456e+00,5.014768e+00,2.920019e-07,6.222807,4.979669,10.237830,5.735366,7.645537,...,5.262205,9.507595,2.273057e+01,4.121429,4.265058,9.976255e+00,4.265606e+00,6.270441e+00,2.273057e+01,3.619051e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.769239,1.593553e+01,1.577140e+01,1.324164e+01,9.976255e+00,5.314437,6.879464,2.742959,6.037669,16.588069,...,13.279606,18.408211,1.582880e+01,7.653594,7.690436,8.429370e-08,1.172983e+01,7.522722e+00,1.582880e+01,9.390729e+00
96,10.115567,5.673473e+00,5.091188e+00,3.496508e+00,4.265606e+00,7.781728,6.288429,11.913096,7.169335,5.487749,...,3.413665,7.604480,2.428997e+01,5.040606,5.515444,1.172983e+01,2.920019e-07,6.432264e+00,2.428997e+01,3.833098e+00
97,6.391769,1.066256e+01,1.045441e+01,8.409672e+00,6.270441e+00,4.339830,4.499079,7.711911,5.427510,11.248370,...,8.410879,12.930078,2.081346e+01,4.323974,4.844911,7.522722e+00,6.432264e+00,1.192093e-07,2.081346e+01,5.756645e+00
98,17.430628,2.815995e+01,2.803783e+01,2.571213e+01,2.273057e+01,19.223271,20.386193,15.825340,19.308610,28.428376,...,25.951277,30.381194,4.768372e-07,20.861901,20.884242,1.582880e+01,2.428997e+01,2.081346e+01,4.768372e-07,2.208806e+01


In [59]:
#sin_df의 인덱스에 영화 제목 대입
sim_df.index = musical_data["title"]
sim_df.columns = musical_data["title"]
sim_df

title,실연남녀,김종욱 찾기,첫사랑,미라클,대장금,렌트,댄싱 섀도우,블루다이아몬드,테너를 빌려줘,스트리트 가이즈,...,펌프보이즈,어린이뮤지컬 퍼니팜,어린이뮤지컬 앤서니 브라운의 돼지책,화성에서 꿈꾸다,우리동네,컨페션,비보이 퍼포먼스 마리오네트,마리아 마리아,어린이뮤지컬 장화신은 고양이,빨간모자2
title,,,,,,,,,,,,,,,,,,,,,
실연남녀,0.000000,1.404393e+01,1.406537e+01,1.145351e+01,8.358102e+00,4.583631,5.386287,3.948157,4.868180,14.675750,...,11.657546,16.399346,1.743063e+01,5.807932,5.930404,3.769239e+00,1.011557e+01,6.391769e+00,1.743063e+01,7.610801e+00
김종욱 찾기,14.043928,6.307963e-07,4.312632e+00,4.948234e+00,7.113935e+00,11.734876,9.899838,16.248913,11.034139,3.090800,...,4.936666,4.718702,2.815995e+01,8.924507,9.474908,1.593553e+01,5.673473e+00,1.066256e+01,2.815995e+01,7.641058e+00
첫사랑,14.065371,4.312632e+00,5.840039e-07,4.519653e+00,6.787456e+00,11.504501,9.867438,15.908099,10.985998,3.159820,...,3.817372,5.055598,2.803783e+01,9.027842,9.068932,1.577140e+01,5.091188e+00,1.045441e+01,2.803783e+01,7.455533e+00
미라클,11.453511,4.948234e+00,4.519653e+00,1.685874e-07,5.014768e+00,9.322559,7.754122,13.568989,8.308853,4.808602,...,3.812373,6.561628,2.571213e+01,6.625292,7.063287,1.324164e+01,3.496508e+00,8.409672e+00,2.571213e+01,5.223513e+00
대장금,8.358102,7.113935e+00,6.787456e+00,5.014768e+00,2.920019e-07,6.222807,4.979669,10.237830,5.735366,7.645537,...,5.262205,9.507595,2.273057e+01,4.121429,4.265058,9.976255e+00,4.265606e+00,6.270441e+00,2.273057e+01,3.619051e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
컨페션,3.769239,1.593553e+01,1.577140e+01,1.324164e+01,9.976255e+00,5.314437,6.879464,2.742959,6.037669,16.588069,...,13.279606,18.408211,1.582880e+01,7.653594,7.690436,8.429370e-08,1.172983e+01,7.522722e+00,1.582880e+01,9.390729e+00
비보이 퍼포먼스 마리오네트,10.115567,5.673473e+00,5.091188e+00,3.496508e+00,4.265606e+00,7.781728,6.288429,11.913096,7.169335,5.487749,...,3.413665,7.604480,2.428997e+01,5.040606,5.515444,1.172983e+01,2.920019e-07,6.432264e+00,2.428997e+01,3.833098e+00
마리아 마리아,6.391769,1.066256e+01,1.045441e+01,8.409672e+00,6.270441e+00,4.339830,4.499079,7.711911,5.427510,11.248370,...,8.410879,12.930078,2.081346e+01,4.323974,4.844911,7.522722e+00,6.432264e+00,1.192093e-07,2.081346e+01,5.756645e+00


In [60]:
# 특정 뮤지컬 작품과의 거리가 가까운 순으로 5개 조회
sim_df["빨간모자2"].sort_values()[1:6]

title
스위니토드       3.327916
화성에서 꿈꾸다    3.376046
피크닉         3.488826
바람의 나라      3.604871
대장금         3.619051
Name: 빨간모자2, dtype: float64

In [ ]:
musical_data.to_csv('../mock_test_csv/musical_data_vector3.csv', index=False, encoding='utf-8')